# Data Cleaning - Initialize Full Ride Data Set

In [2]:
%run setup.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# import modin.pandas as pd

# Load

In [4]:
# city boundaries of washington
washington_boundary = gpd.read_file(config['raw_data_paths']['washington_outline'])
# washington_boundary = gpd.read_file('../data/raw/01_saopaulo_hdi.geojson')
washington_boundary = washington_boundary.drop(columns=['AREAMILES', 'OBJECTID', 'STATE_CITY', 'CAPITAL', 'WEB_URL', 'GLOBALID', 'CREATOR', 'CREATED', 'EDITOR','EDITED', 'SHAPEAREA', 'SHAPELEN'])
washington_boundary.head()

,CITY_NAME,AREAKM,GIS_ID,geometry
0,Washington,177.47,DCBndyPly_1,"POLYGON ((-77.1198 38.93435, -77.11979 38.9343..."


In [ ]:
# try:
#     df = df_orig.copy()
# except:
usecols = ['ride_id', 'member_casual', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng']
dtypes = {
    # 'ride_id'             : str, # not needed
    'rideable_type'       : str,
    'started_at'          : str,
    'ended_at'            : str,
    'start_station_name'  : str,
    'start_station_id'    : str, # does not work here because of inconsistency in data: pd.ArrowDtype(pa.uint32()),
    'end_station_name'    : str,
    'end_station_id'      : str, # does not work here because of inconsistency in data: pd.ArrowDtype(pa.uint32()),
    'start_lat'           : np.float64,
    'start_lng'           : np.float64,
    'end_lat'             : np.float64,
    'end_lng'             : np.float64,
    'member_casual'       : str
    }
df = pd.read_csv(config['raw_data_paths']['full'], usecols=usecols, dtype=dtypes, low_memory=False) #engine='pyarrow')
# df_orig = df.copy()


In [6]:
df.nunique()

ride_id               10693997
rideable_type                3
started_at             9649173
ended_at               9638365
start_station_name         860
start_station_id          1321
end_station_name           865
end_station_id            1338
start_lat               570110
start_lng               617671
end_lat                 173835
end_lng                 183886
member_casual                2
dtype: int64

In [5]:
df.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,classic_bike,2021-01-01 00:08:33,2021-01-01 00:33:53,Maine Ave & 9th St SW,31646.0,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015.0,38.880440,-77.025236,38.894600,-77.072305,member
1,electric_bike,2021-01-01 00:13:43,2021-01-01 00:29:34,10th & U St NW,31111.0,NaN,NaN,38.917193,-77.025894,38.960000,-77.020000,casual
2,electric_bike,2021-01-01 00:14:32,2021-01-01 00:28:45,17th & Corcoran St NW,31214.0,14th & Belmont St NW,31119.0,38.912138,-77.038568,38.920870,-77.031691,member
3,classic_bike,2021-01-01 00:15:45,2021-01-01 00:21:20,Wilson Blvd. & N. Vermont St.,31926.0,Wilson Blvd. & N. Vermont St.,31926.0,38.879477,-77.114563,38.879477,-77.114563,member
4,classic_bike,2021-01-01 00:17:46,2021-01-01 00:21:00,11th & Park Rd NW,31651.0,14th & Newton St NW,31649.0,38.931322,-77.028247,38.931991,-77.032956,member


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10693997 entries, 0 to 10693996
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 1.0+ GB


# NaN Data

In [7]:
df.isna().sum()

rideable_type              0
started_at                 0
ended_at                   0
start_station_name    742776
start_station_id      742776
end_station_name      811282
end_station_id        811282
start_lat                  2
start_lng                  2
end_lat                16765
end_lng                16765
member_casual              0
dtype: int64

- stations: number of data points match, it seems that all bikes that have to been rented oder returned at a station have a missing the entry.
- start_lat, start_lng: every bike usually has a start and end 
- end_lat, end_lng: many, needs investigation 

## Latitude and Longitude

### Start is NaN

In [8]:
mask_start_loc_isna = df.start_lat.isna()
df[mask_start_loc_isna]

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
133129,docked_bike,2021-02-12 17:04:26,2021-02-12 17:12:10,MTL-ECO5-03,MTL-ECO5-03,MTL-ECO5-03,MTL-ECO5-03,NaN,NaN,NaN,NaN,member
143000,docked_bike,2021-02-17 00:54:05,2021-02-17 00:54:17,MTL-ECO5-03,MTL-ECO5-03,MTL-ECO5-03,MTL-ECO5-03,NaN,NaN,NaN,NaN,member


The rides cannot be interpreted anymore. => Drop the entries!

In [9]:
df = df[~mask_start_loc_isna]

### End Coordinates are NaN: Lost Bikes

In [10]:
# check if end_lat, end_lng always appear together
df[(df.end_lat.isna() & ~df.end_lng.isna()) | (~df.end_lat.isna() & df.end_lng.isna())].shape

(0, 12)

In [11]:
mask_lost_bikes = df.end_lat.isna()
lost_bikes = df[mask_lost_bikes]
lost_bikes

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
323,classic_bike,2021-01-01 09:30:20,2021-01-01 11:21:33,Tenleytown / Wisconsin Ave & Albemarle St NW,31303.0,NaN,NaN,38.947607,-77.079382,NaN,NaN,casual
561,classic_bike,2021-01-01 12:57:44,2021-01-01 16:19:10,Lincoln Memorial,31258.0,NaN,NaN,38.888255,-77.049436,NaN,NaN,member
3631,classic_bike,2021-01-02 14:18:26,2021-01-02 15:39:59,Woodglen Dr & Executive Blvd,32078.0,NaN,NaN,39.043170,-77.113500,NaN,NaN,member
3634,classic_bike,2021-01-02 14:18:47,2021-01-02 15:40:08,Woodglen Dr & Executive Blvd,32078.0,NaN,NaN,39.043170,-77.113500,NaN,NaN,member
4170,classic_bike,2021-01-02 14:55:47,2021-01-02 15:13:20,Columbus Circle / Union Station,31623.0,NaN,NaN,38.896960,-77.004930,NaN,NaN,member
...,...,...,...,...,...,...,...,...,...,...,...,...
10691132,classic_bike,2023-12-31 15:52:03,2024-01-01 16:51:59,1st & Q St SW,31676.0,NaN,NaN,38.870824,-77.011915,NaN,NaN,casual
10691183,classic_bike,2023-12-31 15:57:04,2024-01-01 16:56:58,1st & N St SE,31209.0,NaN,NaN,38.875023,-77.005754,NaN,NaN,member
10691725,classic_bike,2023-12-31 16:47:09,2024-01-01 17:47:02,Alabama Ave & Stanton Rd SE / Shops at Park Vi...,31813.0,NaN,NaN,38.846270,-76.981850,NaN,NaN,casual
10693231,classic_bike,2023-12-31 20:33:50,2024-01-01 21:33:36,22nd St & Constitution Ave NW,31292.0,NaN,NaN,38.892441,-77.048947,NaN,NaN,casual


lost bikes: bikes that have been rented, ridden and returned, but could not be located after the ride.

=> drop those entries

In [12]:
df = df[~mask_lost_bikes]

## Analyse Lost Bikes

In [13]:
# data type conversion

In [14]:
# lost_bikes.rideable_type.value_counts()

In [15]:
# duration_lost_rides = (lost_bikes.ended_at - lost_bikes.started_at).dt.total_seconds()
# sns.histplot(x=duration_lost_rides, binrange=[0, 8.999200e+04])

# Convert Data Types

In [16]:
df_before_type_conversion = df.copy()
unique_values_before_type_conversions = df.nunique()

In [17]:
# drop id, reset index
# df = df.drop(columns='ride_id').reset_index()

In [18]:
# Convert categorical columns to category type for memory efficiency
df["rideable_type"]         = df["rideable_type"     ].astype("category")
df["start_station_name"]    = df["start_station_name"].astype("category")
df["end_station_name"]      = df["end_station_name"  ].astype("category")
df["member_casual"]         = df["member_casual"     ].astype("category")

# Convert started_at and ended_at to datetime for memory efficiency
df['started_at']   = pd.to_datetime(df['started_at'])
df['ended_at']     = pd.to_datetime(df['ended_at'  ])

df['started_at']   = pa.Array.from_pandas(df['started_at'], type=pa.timestamp('s'))
df['ended_at']     = pa.Array.from_pandas(df['ended_at'  ], type=pa.timestamp('s'))

df['start_station_id']  = df['start_station_id'].astype('uint32[pyarrow]')
df['end_station_id']    = df['end_station_id'  ].astype('uint32[pyarrow]')


In [19]:
df['start_lat'] = df['start_lat'].round(NDEC_LAT_LNG).astype('float32[pyarrow]') 
df['start_lng'] = df['start_lng'].round(NDEC_LAT_LNG).astype('float32[pyarrow]') 
df['end_lat']   = df['end_lat'  ].round(NDEC_LAT_LNG).astype('float32[pyarrow]') 
df['end_lng']   = df['end_lng'  ].round(NDEC_LAT_LNG).astype('float32[pyarrow]')

In [20]:
unique_values_after_type_conversions = df.nunique()

In [21]:
pd.merge(
    pd.DataFrame(unique_values_after_type_conversions),
    pd.DataFrame(unique_values_before_type_conversions),
    right_index=True, left_index=True
).rename(columns={'0_x' : 'after', '0_y' : 'before'})

,after,before
rideable_type,3,3
started_at,9635493,9635493
ended_at,9624325,9624325
start_station_name,859,859
start_station_id,773,1320
end_station_name,864,864
end_station_id,777,1337
start_lat,13057,570109
start_lng,14653,617670
end_lat,11635,173835


In [22]:
df.dtypes

rideable_type                category
started_at              datetime64[s]
ended_at                datetime64[s]
start_station_name           category
start_station_id      uint32[pyarrow]
end_station_name             category
end_station_id        uint32[pyarrow]
start_lat              float[pyarrow]
start_lng              float[pyarrow]
end_lat                float[pyarrow]
end_lng                float[pyarrow]
member_casual                category
dtype: object

In [23]:
# TODO due to type conversion: did I loose accuracy?

# Export

In [24]:
df.to_parquet(config['processed_data_paths']['rides_init'])